<a href="https://colab.research.google.com/github/IdjiotSandwiches/face-emotion-recognition/blob/data-augmentation/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2 as cv
import albumentations as A
from albumentations.core.composition import OneOf
from albumentations import HorizontalFlip, RandomBrightnessContrast, ShiftScaleRotate
from albumentations import Normalize
from tqdm import tqdm
import numpy as np

d:\PythonProject\env\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
transform = A.Compose([
    A.Resize(48, 48),
    A.RandomBrightnessContrast(p=0.7),
    A.HorizontalFlip(p=0.7),
    A.RandomScale(scale_limit=0.2, p=0.7),
    A.Rotate(limit=15, p=0.7),
])

In [4]:
def augment_and_save(input_dir, output_dir, num_augments=1):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        if(class_name == 'disgust'):
            continue
        class_path = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_path):
            continue 

        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)

        for img_name in tqdm(os.listdir(class_path), desc=f"Processing {class_name}"):
            img_path = os.path.join(class_path, img_name)
            image = cv.imread(img_path)
            if image is None:
                continue
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

            original_output_path = os.path.join(output_class_path, img_name)
            cv.imwrite(original_output_path, cv.cvtColor(image, cv.COLOR_RGB2BGR))

            for i in range(num_augments):
                augmented = transform(image=image)['image']
                aug_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg"
                aug_img_path = os.path.join(output_class_path, aug_img_name)
                cv.imwrite(aug_img_path, cv.cvtColor(augmented, cv.COLOR_RGB2BGR))

In [2]:
def down_sampling(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(input_dir):
        if(class_name == 'disgust'):
            continue

        class_path = f'{input_dir}/{class_name}'
        output_class_path = f'{output_dir}/{class_name}'
        os.makedirs(output_class_path, exist_ok=True)

        for i, img_path in enumerate(tqdm(os.listdir(class_path), desc=f'Processing {class_name}')):
            if(i == 3000):
                break
            img = cv.imread(f'{class_path}/{img_path}')
            
            cv.imwrite(f'{output_class_path}/{img_path}', img)

In [4]:
input_dir = "C:\\Users\\vinar\\Downloads\\FER2013\\train"
output_dir = "C:\\Users\\vinar\\Downloads\\dataset\\FER2013\\train"
down_sampling(input_dir, output_dir)

Processing surprise:  95%|█████████▍| 3000/3171 [00:24<00:01, 123.97it/s]


In [5]:
input_dir = "C:\\Users\\vinar\\Downloads\\FER2013-down_sampling"
output_dir = "C:\\Users\\vinar\\Downloads\\dataset\\FER2013\\train"
augment_and_save(input_dir, output_dir)

Processing angry:   0%|          | 0/3000 [00:00<?, ?it/s]

Processing surprise: 100%|██████████| 3000/3000 [00:04<00:00, 746.50it/s]
